In [1]:
import torch
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.cluster import KMeans
import numpy as np
import random
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [2]:
# Define the image transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def preprocess_image(image):
    # If the image is grayscale, convert it to RGB by duplicating the single channel
    if image.mode != 'RGB':
        image = image.convert('RGB')
    return preprocess(image).unsqueeze(0)  # Add batch dimension


# Visualize the image using matplotlib
def imshow(img_tensor):

    img_tensor = img_tensor.squeeze(0)  # remove the batch dimension
    img_tensor = img_tensor.permute(1, 2, 0)  # change dimensions to (H, W, C)
    img_tensor = img_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])  # un-normalize
    img_tensor = torch.clamp(img_tensor, 0, 1)  # clamp to [0,1]

    plt.imshow(img_tensor)
    plt.axis('off')
    plt.show()

In [5]:
# Load the pre-trained ResNet model
model = models.resnet50(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Load ImageNet class labels
with open("imagenet-classes.txt") as f:
    labels = [line.strip() for line in f.readlines()]


folder_path = "F:/UniOlder/Marburg_Informatik/Project_work/dataset/compressed_rotated"

#folder_path = 'images/images'
file_names = os.listdir(folder_path)
file_names = [f for f in file_names if os.path.isfile(os.path.join(folder_path, f))]


print(f"Found files: {file_names}")

Found files: ['imagenet-classes.txt', 'img-fm1000215.jpg', 'img-fm1001186.jpg', 'img-fm1001570.jpg', 'img-fm1022730.jpg', 'img-fm1022968.jpg', 'img-fm1024045.jpg', 'img-fm1024059.jpg', 'img-fm1037295.jpg', 'img-fm1037916.jpg', 'img-fm1045819.jpg', 'img-fm1048507.jpg', 'img-fm1056947.jpg', 'img-fm1056953.jpg', 'img-fm1060440.jpg', 'img-fm1067477.jpg', 'img-fm1068010.jpg', 'img-fm1068047.jpg', 'img-fm1072311.jpg', 'img-fm1072769.jpg', 'img-fm1072793.jpg', 'img-fm1072852.jpg', 'img-fm1073087.jpg', 'img-fm1073404.jpg', 'img-fm1074501.jpg', 'img-fm1074515.jpg', 'img-fm1078474.jpg', 'img-fm1079107.jpg', 'img-fm1079108.jpg', 'img-fm108.jpg', 'img-fm1081873.jpg', 'img-fm1089589.jpg', 'img-fm1089915.jpg', 'img-fm1090848.jpg', 'img-fm1091242.jpg', 'img-fm1093887.jpg', 'img-fm1093924.jpg', 'img-fm1093925.jpg', 'img-fm1093926.jpg', 'img-fm1093928.jpg', 'img-fm1094519.jpg', 'img-fm1095293.jpg', 'img-fm1099615.jpg', 'img-fm1102392.jpg', 'img-fm1102407.jpg', 'img-fm1102408.jpg', 'img-fm1102410.jpg', 

In [10]:
f

<_io.TextIOWrapper name='imagenet-classes.txt' mode='r' encoding='cp1252'>

In [8]:
# import os

# folder_path = "F:/UniOlder/Marburg_Informatik/Project_work/dataset"
# if os.path.exists(folder_path):
#     print(f"Folder exists: {folder_path}")
# else:
#     print(f"Folder does NOT exist: {folder_path}")


Folder exists: F:/UniOlder/Marburg_Informatik/Project_work/dataset


# ResNET classification - no ground truth

In [ ]:
# fig, axes = plt.subplots(5, 7, figsize=(15, 10))
# axes = axes.flatten()

# random_indexes = random.sample(range(len(file_names)), 35)
# selected_file_names = [file_names[i] for i in random_indexes]

# for idx, image_name in enumerate([file_names[i] for i in random_indexes]):

#     # Load the image
#     image_path = f'{folder_path}/{image_name}'
#     image = Image.open(image_path)

#     # Preprocess the image (handle grayscale to RGB conversion)
#     image_tensor = preprocess_image(image)

#     with torch.no_grad():  # Disable gradient calculation
#         output = model(image_tensor)

#     # Convert output probabilities to predicted class
#     _, predicted_class = output.max(1)

#     # Plot the image in the corresponding subplot
#     img_tensor = image_tensor.squeeze(0).permute(1, 2, 0)  # Remove batch dimension and rearrange to (H, W, C)
#     img_tensor = img_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])  # Un-normalize
#     img_tensor = torch.clamp(img_tensor, 0, 1)  # Clamp to [0,1]

#     # Convert to numpy array for plottAing
#     img_np = img_tensor.numpy()

#     axes[idx].imshow(img_np)
#     axes[idx].axis('off')  # Hide axis
#     axes[idx].set_title(labels[predicted_class.item()], fontsize=8)  # Set the title as the predicted label

# # Hide any remaining subplots if we have fewer images than subplots
# for ax in axes[len(file_names[:35]):]:
#     ax.axis('off')

# plt.tight_layout()
# plt.show()


# ResNET feature extraction and KMeans
https://towardsdatascience.com/how-to-cluster-images-based-on-visual-similarity-cd6e7209fe34

In [6]:
def extract_features(image_tensor):
    with torch.no_grad():
        features = model(image_tensor)
    return features.squeeze().numpy()


def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)


#maybe dbscan

def cluster_images(features_list, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(features_list)
    return kmeans.labels_


In [7]:
# Set the amount of images to process
AMOUNT_OF_IMAGES = 36981 # file_name.len

In [14]:
# # Extract features for each image
# features_list = []

# for file_name in file_names[:AMOUNT_OF_IMAGES]:
#     try:
#         image_tensor = load_image(f'{folder_path}/{file_name}')
#         features = extract_features(image_tensor)
#         features_list.append(features)
#     except Exception as e:
#         print(f'Error processing {folder_path}/{file_name}: {e}')

In [10]:
import os
import pickle

# Folder path to save the file
save_folder = 'NewTest'  # Replace with the folder where you want to save the file
save_file = 'features_list.pkl'  # The name of the file

# Ensure the folder exists
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Full path to save the file
save_path = os.path.join(save_folder, save_file)

# # Save the features_list using pickle
# with open(save_path, 'wb') as f:
#     pickle.dump(features_list, f)

# print(f'Features list saved to {save_path}')

# Load the features_list from the saved file
with open(save_path, 'rb') as f:
    loaded_features_list = pickle.load(f)

print(f'Loaded {len(loaded_features_list)} features.')


Loaded 36981 features.


In [16]:
# features_list

# # # Folder path to save the file
# save_folder = 'emails_by_Prof_Bell'  # Replace with the folder where you want to save the file
# save_file = 'features_list_new.pkl'  # The name of the file

# # # Ensure the folder exists
# if not os.path.exists(save_folder):
#     os.makedirs(save_folder)

# # # Full path to save the file
# save_path = os.path.join(save_folder, save_file)

# # # Save the features_list using pickle
# with open(save_path, 'wb') as f:
#     pickle.dump(features_list, f)

# print(f'Features list saved to {save_path}')



# with open(save_path, 'rb') as f:
#     features_list = pickle.load(f)

# print(f'Loaded {len(features_list)} features.')

Features list saved to emails_by_Prof_Bell\features_list_new.pkl
Loaded 36981 features.


In [11]:
## clustering section
features_array = np.array(loaded_features_list)
num_clusters = 70
clusters = cluster_images(features_array, num_clusters)

E:\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [16]:
# pickle cluster labels

# with open('image_labels.pkl', 'wb') as f:
#     pickle.dump(image_labels, f)
del file_names[0]


file_names

['img-fm1000215.jpg',
 'img-fm1001186.jpg',
 'img-fm1001570.jpg',
 'img-fm1022730.jpg',
 'img-fm1022968.jpg',
 'img-fm1024045.jpg',
 'img-fm1024059.jpg',
 'img-fm1037295.jpg',
 'img-fm1037916.jpg',
 'img-fm1045819.jpg',
 'img-fm1048507.jpg',
 'img-fm1056947.jpg',
 'img-fm1056953.jpg',
 'img-fm1060440.jpg',
 'img-fm1067477.jpg',
 'img-fm1068010.jpg',
 'img-fm1068047.jpg',
 'img-fm1072311.jpg',
 'img-fm1072769.jpg',
 'img-fm1072793.jpg',
 'img-fm1072852.jpg',
 'img-fm1073087.jpg',
 'img-fm1073404.jpg',
 'img-fm1074501.jpg',
 'img-fm1074515.jpg',
 'img-fm1078474.jpg',
 'img-fm1079107.jpg',
 'img-fm1079108.jpg',
 'img-fm108.jpg',
 'img-fm1081873.jpg',
 'img-fm1089589.jpg',
 'img-fm1089915.jpg',
 'img-fm1090848.jpg',
 'img-fm1091242.jpg',
 'img-fm1093887.jpg',
 'img-fm1093924.jpg',
 'img-fm1093925.jpg',
 'img-fm1093926.jpg',
 'img-fm1093928.jpg',
 'img-fm1094519.jpg',
 'img-fm1095293.jpg',
 'img-fm1099615.jpg',
 'img-fm1102392.jpg',
 'img-fm1102407.jpg',
 'img-fm1102408.jpg',
 'img-fm110241

In [17]:
import pandas as pd



# Create a DataFrame
data = {"File Name": file_names, "Cluster": clusters}
df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
output_file = "file_clusters.xlsx"  # Output Excel file name
df.to_excel(output_file, index=False)

print(f"Excel file saved as: {output_file}")

Excel file saved as: file_clusters.xlsx


In [ ]:
for k in range(num_clusters):
    m, n = 11, 7
    N = m*n
    fig, axes = plt.subplots(m, n, figsize=(m*2, 10))
    axes = axes.flatten()

    idx = -1
    # for idx, image_name in enumerate([file_names[i] for i in random_indexes]):
    for image_name, label in zip(file_names[:AMOUNT_OF_IMAGES], clusters):

        if label != k:
            continue

        if idx == N-1:
            break
        else:
            idx += 1

        # Load the image
        image_path = f'{folder_path}/{image_name}'
        try:
            image = Image.open(image_path)

            # Preprocess the image (handle grayscale to RGB conversion)
            image_tensor = preprocess_image(image)

            # Plot the image in the corresponding subplot
            img_tensor = image_tensor.squeeze(0).permute(1, 2, 0)  # Remove batch dimension and rearrange to (H, W, C)
            img_tensor = img_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])  # Un-normalize
            img_tensor = torch.clamp(img_tensor, 0, 1)  # Clamp to [0,1]

            # Convert to numpy array for plottAing
            img_np = img_tensor.numpy()

            axes[idx].imshow(img_np)
            axes[idx].axis('off')  # Hide axis
            axes[idx].set_title(label, fontsize=8)  # Set the title as the predicted label
        except Exception as e:
            print(f'Error processing {image_name}: {e}')

    # Hide any remaining subplots if we have fewer images than subplots
    for ax in axes[len(file_names[:N]):]:
        ax.axis('off')

    plt.tight_layout()
    plt.show()


In [19]:
import os
import pickle
import matplotlib.pyplot as plt
import torch
from PIL import Image

# Folder to save the visualizations
output_folder = "cluster_visualizations"
os.makedirs(output_folder, exist_ok=True)

# Dictionary to store metadata
visualization_metadata = {}

# Loop through each cluster
for k in range(num_clusters):
    m, n = 11, 9
    N = m * n
    fig, axes = plt.subplots(m, n, figsize=(n*3, m*3))
    axes = axes.flatten()

    idx = -1
    for image_name1, label in zip(file_names[:AMOUNT_OF_IMAGES], clusters):
        if label != k:
            continue

        if idx == N - 1:
            break
        else:
            idx += 1

        # Load the image
        image_path1 = os.path.join(folder_path, image_name1)
        try:
            image1 = Image.open(image_path1)

            # Preprocess the image
            image_tensor = preprocess_image(image1)

            # Prepare the image for plotting
            img_tensor = image_tensor.squeeze(0).permute(1, 2, 0)
            img_tensor = img_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])
            img_tensor = torch.clamp(img_tensor, 0, 1)

            # Convert to numpy for plotting
            img_np = img_tensor.numpy()

            axes[idx].imshow(img_np)
            axes[idx].axis('off')
            axes[idx].set_title(label, fontsize=8)
        except Exception as e:
            print(f"Error processing {image_name1}: {e}")

    # Hide unused subplots
    for ax in axes[idx + 1:]:
        ax.axis('off')

    plt.tight_layout()

    
    # Save the figure
    save_path = os.path.join(output_folder, f"cluster_{k}.png")
    plt.savefig(save_path)
   
    plt.close()  # Close the figure to free memory

    # Add metadata for this cluster
    visualization_metadata[k] = save_path

# Save the metadata using pickle
pickle_path = "visualization_metadata1.pkl"
with open(pickle_path, 'wb') as f:
    pickle.dump(visualization_metadata, f)

print(f"Visualization metadata saved to {pickle_path}")





Visualization metadata saved to visualization_metadata1.pkl


In [ ]:
#saving model and extracted features

In [20]:
#to re-arrange the dataset into folders:

import os
import shutil


folder_path
a = 0

output_base_folder = "organized_dataset"  # Path to the output base folder
num_clusters = max(clusters) + 1  # Set to the number of clusters

# Create the output base folder if it doesn't exist
os.makedirs(output_base_folder, exist_ok=True)

# Loop through each cluster and create a corresponding folder
for cluster_id in range(num_clusters):
    cluster_folder = os.path.join(output_base_folder, str(cluster_id))
    os.makedirs(cluster_folder, exist_ok=True)

# Distribute images into corresponding cluster folders
for image_name, cluster_id in zip(file_names, clusters):
    src_path = os.path.join(folder_path, image_name)  # Original image path
    dest_path = os.path.join(output_base_folder, str(cluster_id), image_name)  # Destination path
    try:
        shutil.copy(src_path, dest_path)  # Copy the image
#        print(f"Copied {image_name} to {dest_path}")
    except Exception as e:
        print(f"Error copying {image_name}: {e}")
    a += 1
print("Dataset organized by clusters!")


Copied img-fm1000215.jpg to organized_dataset\34\img-fm1000215.jpg
Copied img-fm1001186.jpg to organized_dataset\44\img-fm1001186.jpg
Copied img-fm1001570.jpg to organized_dataset\51\img-fm1001570.jpg
Copied img-fm1022730.jpg to organized_dataset\22\img-fm1022730.jpg
Copied img-fm1022968.jpg to organized_dataset\22\img-fm1022968.jpg
Copied img-fm1024045.jpg to organized_dataset\58\img-fm1024045.jpg
Copied img-fm1024059.jpg to organized_dataset\68\img-fm1024059.jpg
Copied img-fm1037295.jpg to organized_dataset\15\img-fm1037295.jpg
Copied img-fm1037916.jpg to organized_dataset\58\img-fm1037916.jpg
Copied img-fm1045819.jpg to organized_dataset\68\img-fm1045819.jpg
Copied img-fm1048507.jpg to organized_dataset\15\img-fm1048507.jpg
Copied img-fm1056947.jpg to organized_dataset\1\img-fm1056947.jpg
Copied img-fm1056953.jpg to organized_dataset\44\img-fm1056953.jpg
Copied img-fm1060440.jpg to organized_dataset\34\img-fm1060440.jpg
Copied img-fm1067477.jpg to organized_dataset\55\img-fm1067477.

Copied img-fm121617.jpg to organized_dataset\33\img-fm121617.jpg
Copied img-fm121618.jpg to organized_dataset\56\img-fm121618.jpg
Copied img-fm121619.jpg to organized_dataset\56\img-fm121619.jpg
Copied img-fm121620.jpg to organized_dataset\56\img-fm121620.jpg
Copied img-fm121621.jpg to organized_dataset\56\img-fm121621.jpg
Copied img-fm121622.jpg to organized_dataset\56\img-fm121622.jpg
Copied img-fm121623.jpg to organized_dataset\56\img-fm121623.jpg
Copied img-fm121624.jpg to organized_dataset\56\img-fm121624.jpg
Copied img-fm121625.jpg to organized_dataset\56\img-fm121625.jpg
Copied img-fm121626.jpg to organized_dataset\56\img-fm121626.jpg
Copied img-fm121627.jpg to organized_dataset\56\img-fm121627.jpg
Copied img-fm121628.jpg to organized_dataset\56\img-fm121628.jpg
Copied img-fm121629.jpg to organized_dataset\56\img-fm121629.jpg
Copied img-fm121630.jpg to organized_dataset\68\img-fm121630.jpg
Copied img-fm121631.jpg to organized_dataset\68\img-fm121631.jpg
Copied img-fm121632.jpg t

Copied img-fm121756.jpg to organized_dataset\56\img-fm121756.jpg
Copied img-fm121757.jpg to organized_dataset\56\img-fm121757.jpg
Copied img-fm121758.jpg to organized_dataset\56\img-fm121758.jpg
Copied img-fm121759.jpg to organized_dataset\56\img-fm121759.jpg
Copied img-fm121760.jpg to organized_dataset\56\img-fm121760.jpg
Copied img-fm121761.jpg to organized_dataset\56\img-fm121761.jpg
Copied img-fm121762.jpg to organized_dataset\56\img-fm121762.jpg
Copied img-fm121763.jpg to organized_dataset\56\img-fm121763.jpg
Copied img-fm121764.jpg to organized_dataset\56\img-fm121764.jpg
Copied img-fm121765.jpg to organized_dataset\56\img-fm121765.jpg
Copied img-fm121766.jpg to organized_dataset\56\img-fm121766.jpg
Copied img-fm121767.jpg to organized_dataset\56\img-fm121767.jpg
Copied img-fm121768.jpg to organized_dataset\56\img-fm121768.jpg
Copied img-fm121769.jpg to organized_dataset\56\img-fm121769.jpg
Copied img-fm121770.jpg to organized_dataset\56\img-fm121770.jpg
Copied img-fm121771.jpg t

Copied img-fm131271.jpg to organized_dataset\43\img-fm131271.jpg
Copied img-fm131272.jpg to organized_dataset\21\img-fm131272.jpg
Copied img-fm133099.jpg to organized_dataset\1\img-fm133099.jpg
Copied img-fm1350895.jpg to organized_dataset\1\img-fm1350895.jpg
Copied img-fm1350896.jpg to organized_dataset\19\img-fm1350896.jpg
Copied img-fm1350897.jpg to organized_dataset\1\img-fm1350897.jpg
Copied img-fm1350898.jpg to organized_dataset\1\img-fm1350898.jpg
Copied img-fm1350899.jpg to organized_dataset\19\img-fm1350899.jpg
Copied img-fm1350900.jpg to organized_dataset\68\img-fm1350900.jpg
Copied img-fm1350901.jpg to organized_dataset\57\img-fm1350901.jpg
Copied img-fm1350902.jpg to organized_dataset\32\img-fm1350902.jpg
Copied img-fm1350903.jpg to organized_dataset\19\img-fm1350903.jpg
Copied img-fm1350904.jpg to organized_dataset\33\img-fm1350904.jpg
Copied img-fm1350905.jpg to organized_dataset\15\img-fm1350905.jpg
Copied img-fm1350906.jpg to organized_dataset\49\img-fm1350906.jpg
Copie

Copied img-fm1351165.jpg to organized_dataset\49\img-fm1351165.jpg
Copied img-fm1351166.jpg to organized_dataset\49\img-fm1351166.jpg
Copied img-fm1351167.jpg to organized_dataset\49\img-fm1351167.jpg
Copied img-fm1351168.jpg to organized_dataset\55\img-fm1351168.jpg
Copied img-fm1351170.jpg to organized_dataset\55\img-fm1351170.jpg
Copied img-fm1351171.jpg to organized_dataset\6\img-fm1351171.jpg
Copied img-fm1351173.jpg to organized_dataset\6\img-fm1351173.jpg
Copied img-fm1351174.jpg to organized_dataset\7\img-fm1351174.jpg
Copied img-fm1351175.jpg to organized_dataset\23\img-fm1351175.jpg
Copied img-fm1351176.jpg to organized_dataset\23\img-fm1351176.jpg
Copied img-fm1351177.jpg to organized_dataset\19\img-fm1351177.jpg
Copied img-fm1351178.jpg to organized_dataset\1\img-fm1351178.jpg
Copied img-fm1351179.jpg to organized_dataset\7\img-fm1351179.jpg
Copied img-fm1351180.jpg to organized_dataset\57\img-fm1351180.jpg
Copied img-fm1351181.jpg to organized_dataset\57\img-fm1351181.jpg


...

Copied scn-mi12671a14.jpg to organized_dataset\5\scn-mi12671a14.jpg
Copied scn-mi12671b01.jpg to organized_dataset\27\scn-mi12671b01.jpg
Copied scn-mi12671b02.jpg to organized_dataset\5\scn-mi12671b02.jpg
Copied scn-mi12671b03.jpg to organized_dataset\5\scn-mi12671b03.jpg
Copied scn-mi12671b04.jpg to organized_dataset\14\scn-mi12671b04.jpg
Copied scn-mi12671b05.jpg to organized_dataset\38\scn-mi12671b05.jpg
Copied scn-mi12671b06.jpg to organized_dataset\38\scn-mi12671b06.jpg
Copied scn-mi12671b07.jpg to organized_dataset\5\scn-mi12671b07.jpg
Copied scn-mi12671b08.jpg to organized_dataset\38\scn-mi12671b08.jpg
Copied scn-mi12671b09.jpg to organized_dataset\66\scn-mi12671b09.jpg
Copied scn-mi12671b10.jpg to organized_dataset\66\scn-mi12671b10.jpg
Copied scn-mi12671b11.jpg to organized_dataset\66\scn-mi12671b11.jpg
Copied scn-mi12671b12.jpg to organized_dataset\27\scn-mi12671b12.jpg
Copied scn-mi12671b13.jpg to organized_dataset\27\scn-mi12671b13.jpg
Copied scn-mi12671b14.jpg to organized

Copied scn-mi12673c08.jpg to organized_dataset\38\scn-mi12673c08.jpg
Copied scn-mi12673c09.jpg to organized_dataset\38\scn-mi12673c09.jpg
Copied scn-mi12673c10.jpg to organized_dataset\5\scn-mi12673c10.jpg
Copied scn-mi12673c11.jpg to organized_dataset\38\scn-mi12673c11.jpg
Copied scn-mi12673c12.jpg to organized_dataset\38\scn-mi12673c12.jpg
Copied scn-mi12673c13.jpg to organized_dataset\16\scn-mi12673c13.jpg
Copied scn-mi12673c14.jpg to organized_dataset\38\scn-mi12673c14.jpg
Copied scn-mi12673d01.jpg to organized_dataset\38\scn-mi12673d01.jpg
Copied scn-mi12673d02.jpg to organized_dataset\67\scn-mi12673d02.jpg
Copied scn-mi12673d03.jpg to organized_dataset\38\scn-mi12673d03.jpg
Copied scn-mi12673d04.jpg to organized_dataset\5\scn-mi12673d04.jpg
Copied scn-mi12673d05.jpg to organized_dataset\5\scn-mi12673d05.jpg
Copied scn-mi12673d06.jpg to organized_dataset\5\scn-mi12673d06.jpg
Copied scn-mi12673d07.jpg to organized_dataset\5\scn-mi12673d07.jpg
Copied scn-mi12673d08.jpg to organized_

Copied scn-mi12674g10.jpg to organized_dataset\62\scn-mi12674g10.jpg
Copied scn-mi12674g11.jpg to organized_dataset\35\scn-mi12674g11.jpg
Copied scn-mi12674g12.jpg to organized_dataset\38\scn-mi12674g12.jpg
Copied scn-mi12674g13.jpg to organized_dataset\65\scn-mi12674g13.jpg
Copied scn-mi12674g14.jpg to organized_dataset\14\scn-mi12674g14.jpg
Copied scn-mi12675a01.jpg to organized_dataset\62\scn-mi12675a01.jpg
Copied scn-mi12675a02.jpg to organized_dataset\14\scn-mi12675a02.jpg
Copied scn-mi12675a03.jpg to organized_dataset\14\scn-mi12675a03.jpg
Copied scn-mi12675a04.jpg to organized_dataset\14\scn-mi12675a04.jpg
Copied scn-mi12675a05.jpg to organized_dataset\27\scn-mi12675a05.jpg
Copied scn-mi12675a06.jpg to organized_dataset\27\scn-mi12675a06.jpg
Copied scn-mi12675a07.jpg to organized_dataset\14\scn-mi12675a07.jpg
Copied scn-mi12675a08.jpg to organized_dataset\38\scn-mi12675a08.jpg
Copied scn-mi12675a09.jpg to organized_dataset\43\scn-mi12675a09.jpg
Copied scn-mi12675a10.jpg to organ

Copied scn-mi12958f14.jpg to organized_dataset\3\scn-mi12958f14.jpg
Copied scn-mi12958g01.jpg to organized_dataset\59\scn-mi12958g01.jpg
Copied scn-mi12960d11.jpg to organized_dataset\36\scn-mi12960d11.jpg
Copied scn-mi12960d12.jpg to organized_dataset\36\scn-mi12960d12.jpg
Copied scn-mi12968f12.jpg to organized_dataset\37\scn-mi12968f12.jpg
Copied scn-mi12968g02.jpg to organized_dataset\5\scn-mi12968g02.jpg
Copied scn-mi12969a02.jpg to organized_dataset\37\scn-mi12969a02.jpg
Copied scn-mi12969a05.jpg to organized_dataset\5\scn-mi12969a05.jpg
Copied scn-mi12969b06.jpg to organized_dataset\66\scn-mi12969b06.jpg
Copied scn-mi12969b07.jpg to organized_dataset\66\scn-mi12969b07.jpg
Copied scn-mi12969b08.jpg to organized_dataset\31\scn-mi12969b08.jpg
Copied scn-mi12969c02.jpg to organized_dataset\5\scn-mi12969c02.jpg
Copied scn-mi12969c03.jpg to organized_dataset\66\scn-mi12969c03.jpg
Copied scn-mi12969c09.jpg to organized_dataset\66\scn-mi12969c09.jpg
Copied scn-mi12969c14.jpg to organized

Copied scn-nl00002f10.jpg to organized_dataset\67\scn-nl00002f10.jpg
Copied scn-nl00002f11.jpg to organized_dataset\12\scn-nl00002f11.jpg
Copied scn-nl00002f12.jpg to organized_dataset\4\scn-nl00002f12.jpg
Copied scn-nl00002f13.jpg to organized_dataset\31\scn-nl00002f13.jpg
Copied scn-nl00030f13.jpg to organized_dataset\59\scn-nl00030f13.jpg
Copied scn-nl00030f14.jpg to organized_dataset\59\scn-nl00030f14.jpg
Copied scn-nl00077e10.jpg to organized_dataset\59\scn-nl00077e10.jpg
Copied scn-oe00017g06.jpg to organized_dataset\29\scn-oe00017g06.jpg
Copied scn-oe00024f11.jpg to organized_dataset\28\scn-oe00024f11.jpg
Copied scn-oe00032e14.jpg to organized_dataset\28\scn-oe00032e14.jpg
Copied scn-oe00112b04.jpg to organized_dataset\67\scn-oe00112b04.jpg
Copied scn-oe00114c06.jpg to organized_dataset\67\scn-oe00114c06.jpg
Copied scn-oe00138c09.jpg to organized_dataset\36\scn-oe00138c09.jpg
Copied scn-oe00157f05.jpg to organized_dataset\67\scn-oe00157f05.jpg
Copied scn-oe00171d08.jpg to organi

In [25]:
len_clusters = [0]*num_clusters 
for c in clusters:
    len_clusters[c] += 1 

        
len_clusters

[1035,
 685,
 740,
 638,
 563,
 1239,
 403,
 518,
 218,
 934,
 431,
 458,
 620,
 141,
 470,
 570,
 384,
 343,
 616,
 364,
 433,
 604,
 238,
 593,
 601,
 535,
 789,
 985,
 635,
 582,
 325,
 597,
 490,
 302,
 446,
 539,
 836,
 434,
 861,
 599,
 389,
 360,
 421,
 1173,
 279,
 254,
 323,
 261,
 435,
 412,
 441,
 527,
 298,
 328,
 454,
 491,
 521,
 516,
 417,
 769,
 440,
 370,
 367,
 335,
 674,
 334,
 880,
 922,
 368,
 428]

# till here

In [23]:
# pca = PCA(n_components=2)
# reduced_features = pca.fit_transform(features_array)
tsne = TSNE(n_components=2, random_state=42)
reduced_features = tsne.fit_transform(features_array)

KeyboardInterrupt: 

In [ ]:
def imscatter(x, y, image_paths, ax=None, zoom=0.1):
    if ax is None:
        ax = plt.gca()
    pictures = []
    for i, image_path in enumerate(image_paths):
        try:
            img = plt.imread(folder_path+'/'+image_path)
            if len(img.shape) == 2:  # grayscale
                im = OffsetImage(img, zoom=zoom, cmap='gray')
            else:  # color
                im = OffsetImage(img, zoom=zoom)
            ab = AnnotationBbox(im, (x[i], y[i]), xycoords='data', frameon=False)
            pictures.append(ax.add_artist(ab))
            ax.text(x[i], y[i] + 1.5, clusters[i], color='black', fontsize=9, ha='center')
        except Exception as e:
            print(f"Error loading image {folder_path+'/'+image_path}: {e}")
    return pictures



fig, ax = plt.subplots(figsize=(30, 20))
x_vals = reduced_features[:, 0]
y_vals = reduced_features[:, 1]

IMAGES_TO_PLOT=1500
imscatter(x_vals[:IMAGES_TO_PLOT], y_vals[:IMAGES_TO_PLOT], file_names[:IMAGES_TO_PLOT], ax=ax, zoom=0.025)
# imscatter(x_vals, y_vals, file_names[:AMOUNT_OF_IMAGES], ax=ax, zoom=0.025)

ax.set_title('Images clustering')
plt.xlim(min(x_vals)+10, max(x_vals)+10)
plt.ylim(min(y_vals)+10, max(y_vals)+10)
plt.show()

In [57]:
num_clusters

100

# TURTLE clusterisation
The code is built with the following libraries

PyTorch - 2.2.1  
torchvision - 0.17.1  
numpy  
scipy  
scikit-learn  
clip  
tqdm  
cuml - 24.02


In [ ]:
wget https://brbiclab.epfl.ch/wp-content/uploads/2024/06/turtle_tasks.zip


"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [ ]:
# pip create -n rapids-24.08 -c rapidsai -c conda-forge -c nvidia rapids=24.08 python=3.11 'cuda-version>=12.0,<=12.5'

Note: you may need to restart the kernel to use updated packages.


ЌҐ г¤ Ґвбп ­ ©вЁ гЄ § ­­л© д ©«.


# Transfer learning
https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/